# Epic 4: Lookalikes

## Imports

In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

import joblib
import pickle

import warnings
warnings.filterwarnings('ignore')


np.random.seed(42)

## Get data

In [2]:
# Read in data
df = pd.read_csv('data/data_new.csv')
df

,crm_contact_functietitel,crm_contact_id,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,crm_persoon_mail_thema_innovatie,crm_persoon_mail_thema_internationaal_ondernemen,crm_persoon_mail_thema_mobiliteit,crm_persoon_mail_thema_omgeving,crm_persoon_mail_thema_sales_marketing_communicatie,crm_persoon_mail_thema_strategie_en_algemeen_management,...,crm_activiteitscode_naam_Papier & karton,crm_activiteitscode_naam_Technologische industrie & diensten,crm_activiteitscode_naam_Telecom & IT,"crm_activiteitscode_naam_Textiel, kleding en confectie",crm_activiteitscode_naam_Vastgoed,crm_activiteitscode_naam_Verenigingen en maatschappelijke organisaties,crm_activiteitscode_naam_Verzekering,crm_activiteitscode_naam_Voeding,crm_activiteitscode_naam_Vrije beroepen,crm_activiteitscode_naam_Zorg
0,14438,B05C5B73-4F93-E511-B7D1-005056B06EC4,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1887,CB54A5ED-0D3F-ED11-9DB0-6045BD895BFB,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14438,4B95BAFD-ABFA-E611-80E4-001DD8B72B62,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2383,80E226C4-AFC5-E911-8104-001DD8B72B61,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14438,9DC0BE41-64B2-E711-80EC-001DD8B72B62,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55481,2678,DD2FAD8A-8CEF-E611-80E4-001DD8B72B62,0,1,0,1,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55482,6035,8D1F28C1-0673-E111-B43A-00505680000A,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55483,12243,F3025D6B-C95E-EA11-810F-001DD8B72B62,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55484,749,CE6EF7AA-AD72-E111-B43A-00505680000A,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Get the id column
id_col = df['crm_contact_id']
id_col

0        B05C5B73-4F93-E511-B7D1-005056B06EC4
1        CB54A5ED-0D3F-ED11-9DB0-6045BD895BFB
2        4B95BAFD-ABFA-E611-80E4-001DD8B72B62
3        80E226C4-AFC5-E911-8104-001DD8B72B61
4        9DC0BE41-64B2-E711-80EC-001DD8B72B62
                         ...                 
55481    DD2FAD8A-8CEF-E611-80E4-001DD8B72B62
55482    8D1F28C1-0673-E111-B43A-00505680000A
55483    F3025D6B-C95E-EA11-810F-001DD8B72B62
55484    CE6EF7AA-AD72-E111-B43A-00505680000A
55485    C42ECD47-066C-E111-B43A-00505680000A
Name: crm_contact_id, Length: 55486, dtype: object

In [4]:
# Get the data without the id column
df_clipped = df.drop(['crm_contact_id'], axis=1)
df_clipped

,crm_contact_functietitel,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,crm_persoon_mail_thema_innovatie,crm_persoon_mail_thema_internationaal_ondernemen,crm_persoon_mail_thema_mobiliteit,crm_persoon_mail_thema_omgeving,crm_persoon_mail_thema_sales_marketing_communicatie,crm_persoon_mail_thema_strategie_en_algemeen_management,crm_Persoon_Mail_thema_talent,...,crm_activiteitscode_naam_Papier & karton,crm_activiteitscode_naam_Technologische industrie & diensten,crm_activiteitscode_naam_Telecom & IT,"crm_activiteitscode_naam_Textiel, kleding en confectie",crm_activiteitscode_naam_Vastgoed,crm_activiteitscode_naam_Verenigingen en maatschappelijke organisaties,crm_activiteitscode_naam_Verzekering,crm_activiteitscode_naam_Voeding,crm_activiteitscode_naam_Vrije beroepen,crm_activiteitscode_naam_Zorg
0,14438,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1887,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14438,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2383,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14438,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55481,2678,0,1,0,1,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55482,6035,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55483,12243,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55484,749,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Use the imputer from ./data/imputer.pkl
with open('data/imputer.pkl', 'rb') as f:
    imputer = pickle.load(f)

# Fit the imputer on the dataframe (not the id column)
imputer.fit(df_clipped)

# Transform the dataframe
df_clipped = pd.DataFrame(imputer.transform(df_clipped), columns=df_clipped.columns)
df_clipped

,crm_contact_functietitel,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,crm_persoon_mail_thema_innovatie,crm_persoon_mail_thema_internationaal_ondernemen,crm_persoon_mail_thema_mobiliteit,crm_persoon_mail_thema_omgeving,crm_persoon_mail_thema_sales_marketing_communicatie,crm_persoon_mail_thema_strategie_en_algemeen_management,crm_Persoon_Mail_thema_talent,...,crm_activiteitscode_naam_Papier & karton,crm_activiteitscode_naam_Technologische industrie & diensten,crm_activiteitscode_naam_Telecom & IT,"crm_activiteitscode_naam_Textiel, kleding en confectie",crm_activiteitscode_naam_Vastgoed,crm_activiteitscode_naam_Verenigingen en maatschappelijke organisaties,crm_activiteitscode_naam_Verzekering,crm_activiteitscode_naam_Voeding,crm_activiteitscode_naam_Vrije beroepen,crm_activiteitscode_naam_Zorg
0,14438.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1887.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14438.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2383.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14438.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55481,2678.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55482,6035.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55483,12243.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55484,749.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Scale the data (all columns but not the crm_contact_id column)
with open('data/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Fit the scaler on the dataframe (not the id column)
scaler.fit(df_clipped)

# Transform the DataFrame by replacing NaN values with the computed values
df_scaled = scaler.transform(df_clipped)

# The result is a NumPy array, so you can convert it back to a DataFrame if needed
df_scaled = pd.DataFrame(df_scaled, columns=df_clipped.columns)
df_scaled

,crm_contact_functietitel,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,crm_persoon_mail_thema_innovatie,crm_persoon_mail_thema_internationaal_ondernemen,crm_persoon_mail_thema_mobiliteit,crm_persoon_mail_thema_omgeving,crm_persoon_mail_thema_sales_marketing_communicatie,crm_persoon_mail_thema_strategie_en_algemeen_management,crm_Persoon_Mail_thema_talent,...,crm_activiteitscode_naam_Papier & karton,crm_activiteitscode_naam_Technologische industrie & diensten,crm_activiteitscode_naam_Telecom & IT,"crm_activiteitscode_naam_Textiel, kleding en confectie",crm_activiteitscode_naam_Vastgoed,crm_activiteitscode_naam_Verenigingen en maatschappelijke organisaties,crm_activiteitscode_naam_Verzekering,crm_activiteitscode_naam_Voeding,crm_activiteitscode_naam_Vrije beroepen,crm_activiteitscode_naam_Zorg
0,0.937046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.122469,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.937046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.154660,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.937046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55481,0.173806,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
55482,0.391680,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
55483,0.794587,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
55484,0.048611,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [7]:
# Set the ID column back
df = pd.concat([id_col, df_scaled], axis=1)
df

,crm_contact_id,crm_contact_functietitel,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,crm_persoon_mail_thema_innovatie,crm_persoon_mail_thema_internationaal_ondernemen,crm_persoon_mail_thema_mobiliteit,crm_persoon_mail_thema_omgeving,crm_persoon_mail_thema_sales_marketing_communicatie,crm_persoon_mail_thema_strategie_en_algemeen_management,...,crm_activiteitscode_naam_Papier & karton,crm_activiteitscode_naam_Technologische industrie & diensten,crm_activiteitscode_naam_Telecom & IT,"crm_activiteitscode_naam_Textiel, kleding en confectie",crm_activiteitscode_naam_Vastgoed,crm_activiteitscode_naam_Verenigingen en maatschappelijke organisaties,crm_activiteitscode_naam_Verzekering,crm_activiteitscode_naam_Voeding,crm_activiteitscode_naam_Vrije beroepen,crm_activiteitscode_naam_Zorg
0,B05C5B73-4F93-E511-B7D1-005056B06EC4,0.937046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,CB54A5ED-0D3F-ED11-9DB0-6045BD895BFB,0.122469,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,4B95BAFD-ABFA-E611-80E4-001DD8B72B62,0.937046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,80E226C4-AFC5-E911-8104-001DD8B72B61,0.154660,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,9DC0BE41-64B2-E711-80EC-001DD8B72B62,0.937046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55481,DD2FAD8A-8CEF-E611-80E4-001DD8B72B62,0.173806,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
55482,8D1F28C1-0673-E111-B43A-00505680000A,0.391680,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
55483,F3025D6B-C95E-EA11-810F-001DD8B72B62,0.794587,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
55484,CE6EF7AA-AD72-E111-B43A-00505680000A,0.048611,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [8]:
# Save df to csv
df.to_csv('data/data_scaled.csv', index=False)

## Model

### Functions to train, save & load the model

In [9]:
def train_kmeans_model(X):
    # Create variables for the model
    id_column = "crm_contact_id"
    k = 80
    X = X.drop(columns=[id_column])

    # Create a kmeans model based on the data
    kmeans_model = KMeans(n_clusters=k, random_state=42).fit(X)

    # Return the model
    return kmeans_model

In [10]:
def save_kmeans_model(model, filename='./data/kmeans_model.joblib'):
    # Save the KMeans model to a file
    joblib.dump(model, filename)

In [11]:
def load_kmeans_model(filename='./data/kmeans_model.joblib'):
    # Load the KMeans model from a file
    return joblib.load(filename)

In [12]:
kmeans_model = train_kmeans_model(df)
save_kmeans_model(kmeans_model)

### Function to find the lookalikes

In [13]:
def find_lookalikes(data, selected_id, top_n=10):
    id_column = "crm_contact_id"

    # load the model
    kmeans_model = load_kmeans_model()

    # Add cluster labels to the original dataset
    data['Cluster'] = kmeans_model.labels_

    # Select a row based on the specified ID
    selected_row = data[data[id_column] == selected_id]

    # Check if selected_row is not empty
    if not selected_row.empty:
        # Get the cluster label of the selected row
        selected_cluster = selected_row['Cluster'].values[0]

        # Filter the dataset to get all points in the same cluster
        cluster_points = data[data['Cluster'] == selected_cluster]

        # Calculate the distance from the selected point to all points in the cluster
        distances = np.linalg.norm(cluster_points.drop(columns=['Cluster', id_column]) - selected_row.drop(columns=['Cluster', id_column]), axis=1)

        # Combine distances with the cluster_points DataFrame
        cluster_points['Distance'] = distances

        # Sort by distance and select the top N
        top_lookalikes = cluster_points.sort_values(by='Distance').head(top_n + 1)

        # Remove the cluster label and distance to avoid disrupting the original dataset
        data = data.drop(columns=['Cluster'])
        top_lookalikes = top_lookalikes.drop(columns=['Cluster', 'Distance'])

        # Filter the id
        top_lookalikes = top_lookalikes['crm_contact_id']

        # skip the first row because it is the same as the selected row
        top_lookalikes = top_lookalikes[1:]

        # format the output
        top_lookalikes.to_csv('data/top_lookalikes.csv', index=False)

        return top_lookalikes
    else:
        # Handle the case where selected_row is empty
        print(f"No row found with {id_column} equal to {selected_id}")
        return None

### Inputs for the lookalikes function

In [14]:
# Select an ID to find lookalikes
selected_id = "8D1F28C1-0673-E111-B43A-00505680000A"

# Call the function to find top X lookalikes
top_lookalikes = find_lookalikes(df, selected_id, top_n=15)